# Quantitative Momentum Strategy.

Investing in the stocks that have increased in price the most.

In [1]:
# Import libraries

import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore
import xlsxwriter

In [2]:
# Import list of stocks

stocks = pd.read_csv("sp_500_stocks.csv")
stocks = stocks.drop([135,219, 467, 484]) # stocks giving error, might not be in list
# print(stocks)

In [3]:
# Import IEX Cloud API Token (Sandbox API)

from secret import IEX_CLOUD_API_TOKEN
token = IEX_CLOUD_API_TOKEN

In [4]:
# Make first API Call
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={token}"
data = requests.get(api_url).json()
# data

# Parse API
# data['year1ChangePercent']
data

{'companyName': 'Apple Inc',
 'marketcap': 1998975633166,
 'week52high': 183.8,
 'week52low': 126.05,
 'week52highSplitAdjustOnly': 182.97,
 'week52lowSplitAdjustOnly': 124.58,
 'week52change': -0.3138330379935121,
 'sharesOutstanding': 16407613206,
 'float': 0,
 'avg10Volume': 84352967,
 'avg30Volume': 83609799,
 'day200MovingAvg': 150.66,
 'day50MovingAvg': 146.29,
 'employees': 150486,
 'ttmEPS': 6.3,
 'ttmDividendRate': 1.105966787957242,
 'dividendYield': 0.008629913431795595,
 'nextDividendDate': '',
 'exDividendDate': '2022-12-15',
 'nextEarningsDate': '2023-01-14',
 'peRatio': 20.448700825230635,
 'beta': 1.3446524792852397,
 'maxChangePercent': 49.04339712643232,
 'year5ChangePercent': 2.08840796950289,
 'year2ChangePercent': -0.04652480995778121,
 'year1ChangePercent': -0.31878305582264,
 'ytdChangePercent': -0.03886132011497364,
 'month6ChangePercent': -0.09814169701558863,
 'month3ChangePercent': -0.11951674515904048,
 'month1ChangePercent': -0.15658140790334274,
 'day30Cha

## Execute Batch API Call and Build Dataframe.

In [5]:
def divide_chunks(lst, n):     
    """Yield successive n sized chunks/batches from lst"""
    for i in range(0, len(lst), n):
        yield lst.loc[i:i + n]
        
symbol_groups = list(divide_chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range (0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of shares to buy']

In [6]:
final_df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    # Make Batch API Call 
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,quote&token={token}"
    response = requests.get(batch_api_call_url)
    response.raise_for_status()  # raises exception when not a 2xx response
    if response.status_code != 204:
        data = response.json()
#     print(data)

    for symbol in symbol_string.split(','):
        
        if symbol in data.keys():
            # Parse API Call
            price = data[symbol]['quote']['latestPrice']
            oneYearReturn = data[symbol]['stats']['year1ChangePercent']

            # Add stocks to dataframe
            new_stock = pd.DataFrame([[symbol, price, oneYearReturn, "N/A"]], columns = my_columns)
            final_df = pd.concat([final_df, new_stock])

In [7]:
final_df = final_df.set_index("Ticker").reset_index()
final_df

,Ticker,Price,One-Year Price Return,Number of shares to buy
0,A,156.56,-0.035047,N/A
1,AAL,12.84,-0.32072,N/A
2,AAP,152.14,-0.349549,N/A
3,AAPL,130.96,-0.309604,N/A
4,ABBV,165.42,0.250971,N/A
...,...,...,...,...
499,YUM,128.50,-0.054744,N/A
500,ZBH,132.06,-0.006342,N/A
501,ZBRA,265.90,-0.56156,N/A
502,ZION,50.46,-0.233169,N/A


## Removing Low-Momentum Stocks.

In [8]:
final_df = final_df.sort_values('One-Year Price Return', ascending = False)[:50].reset_index()
# final_df

## Calculating number of shares to buy.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the value of your portfolio:')

    try:
        val = float(portfolio_size)
    except ValueError:
        print("Please enter an integer.")
        portfolio_size = input('Enter the value of your portfolio:')
        val = float(portfolio_size)

In [10]:
portfolio_input()

Enter the value of your portfolio:1000000


In [11]:
position_size = float(portfolio_size)/len(final_df.index)
final_df['Number of shares to buy'] = (position_size/final_df['Price']).apply(np.floor)
# final_df

## Building a better and more realistic Momentum Strategy.

Differentiate between high quality (consistent, stable growth) momentum stocks vs low quality (inconsistent, sharp growth) momentum stocks. 

We will select stocks from the highest percentiles of:
* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

In [17]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of shares to buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_df = pd.DataFrame(columns = hqm_columns)
# hqm_df

In [18]:
for symbol_string in symbol_strings[:1]:
    # Make Batch API Call 
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,quote&token={token}"
    response = requests.get(batch_api_call_url)
    response.raise_for_status()  # raises exception when not a 2xx response
    if response.status_code != 204:
        data = response.json()
    
    for symbol in symbol_string.split(','):
        
        if symbol in data.keys():
            # Parse API Call
            price = data[symbol]['quote']['latestPrice']
            oneYearPriceReturn = data[symbol]['stats']['year1ChangePercent']
            sixMonthPriceReturn = data[symbol]['stats']['month6ChangePercent']
            threeMonthPriceReturn = data[symbol]['stats']['month3ChangePercent']
            oneMonthPriceReturn = data[symbol]['stats']['month1ChangePercent']

            # Add stocks to dataframe
            new_stock = pd.DataFrame([[
                symbol, 
                price, 
                "N/A", 
                oneYearPriceReturn,
                "N/A",
                sixMonthPriceReturn, 
                "N/A",
                threeMonthPriceReturn, 
                "N/A",
                oneMonthPriceReturn,
                "N/A",
                "N/A"
            ]], columns = hqm_columns)
            hqm_df = pd.concat([hqm_df, new_stock])

In [35]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_df.index:
    for time_period in time_periods:
        hqm_df.loc[row, f'{time_period} Return Percentile'] = percentileofscore(hqm_df[f'{time_period} Price Return'], hqm_df.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_df[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
# hqm_df

## Calculating HQM Scores.

In [34]:
from statistics import mean
hqm_df['HQM Score'] = (hqm_df['One-Year Return Percentile'] 
                       + hqm_df['Six-Month Return Percentile'] 
                       + hqm_df['Three-Month Return Percentile'] 
                       + hqm_df['One-Month Return Percentile'])/4

# hqm_df

## Select 50 Best Momentum Stocks.

In [33]:
hqm_df.sort_values(by = 'HQM Score', ascending = False)
hqm_df = hqm_df[:51]
# hqm_df

## Calculate number of shares to buy.

In [36]:
portfolio_input()

Enter the value of your portfolio:1000000


In [39]:
position_size = float(portfolio_size)/len(hqm_df.index)
hqm_df['Number of shares to buy'] = (position_size/hqm_df['Price']).apply(np.floor)
# hqm_df

## Format Excel Output.

In [41]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [42]:
# Set Formats

background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [43]:
# Apply formats
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of shares to buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [44]:
# Save .xlsx file
writer.save()

<ipython-input-44-4cfb2f925781>:2: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
